In [ ]:
from IPython.display import display, clear_output
import ipywidgets as ipw
import widgets
import utils
import os
import json
import shutil

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
DATA_MODEL = utils.read_yaml("/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])
slabs_options = [DATA_MODEL["classes"][object_type]["title"] for object_type in CONFIG["slabs_concepts"]]
slabs_options.insert(0, "No material")

material_selection_radio_button = utils.Radiobuttons(
    description = '', options = slabs_options, 
    disabled = False, layout = ipw.Layout(width = '300px'), 
    style = {'description_width': "100px"}
)

material_selector = widgets.MaterialSelectionWidget()

molecule_selector_list = widgets.MultipleSelectorWidget("molecule_concept")
molecule_selector_list_output = ipw.Output()

add_molecule_button = utils.Button(
    description = 'Add molecule concept', disabled = False, button_style = '', 
    tooltip = 'Add molecule concept', layout = ipw.Layout(width = '150px', height = '25px')
)
remove_molecule_button = utils.Button(
    description = 'Remove molecule concept', disabled = False, button_style = '', 
    tooltip = 'Remove molecule concept', layout = ipw.Layout(width = '150px', height = '25px')
)
add_remove_molecule_buttons_hbox = ipw.HBox([add_molecule_button, remove_molecule_button])

product_widgets = widgets.OpenbisObjectWidget("ReactionProductConcept", DATA_MODEL)

product_support_files = ipw.FileUpload(multiple = True)

product_cdxml = ipw.FileUpload(multiple = False, accept = '.cdxml')

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def create_product_openbis(b):
    if not product_cdxml.value:
        display(utils.Javascript(data = "alert('Upload a CDXML file.')"))
        return
    
    molecule_concepts = []
    if molecule_selector_list.selectors:
        for molecule_selector in molecule_selector_list.selectors:
            if molecule_selector.dropdown.value != -1:
                molecule_concepts.append(molecule_selector.dropdown.value)
        
    object_properties = utils.get_object_widget_values(product_widgets.properties_widgets)
    
    if molecule_concepts:
        object_properties["molecule_concepts"] = molecule_concepts
    
    if material_selector.dropdown.value != -1:
        object_properties["crystal_concept"] = material_selector.dropdown.value

    # Check if the product is already in openBIS
    products_openbis = [f"{product.props['name']}" for product in utils.get_openbis_objects(OPENBIS_SESSION, type ="REACTION_PRODUCT_CONCEPT")]
    
    product_name = object_properties['name']
    if product_name in products_openbis:
        display(utils.Javascript(data = "alert('Reaction product concept is already in openBIS!')"))
        return

    product_object = utils.create_openbis_object(
        OPENBIS_SESSION, type="REACTION_PRODUCT_CONCEPT",
        collection="/MATERIALS/MOLECULES/PRODUCT_COLLECTION", 
        props = object_properties
    )
    utils.upload_datasets(OPENBIS_SESSION, product_object, product_cdxml, "RAW_DATA")
    utils.upload_datasets(OPENBIS_SESSION, product_object, product_support_files, "RAW_DATA")
    display(utils.Javascript(data = "alert('Upload successful!')"))

def checkbox_on_change(checkbox, textbox):
    def enable_textbox(change):
        if checkbox.value:
            textbox.disabled = False
        else:
            textbox.disabled = True
    checkbox.observe(enable_textbox, names = "value")

def add_molecule_widget(b):
    molecule_selector_list.add_selector()
    with molecule_selector_list_output:
        clear_output()
        display(molecule_selector_list)

def remove_molecule_widget(b):
    molecule_selector_list.remove_selector()
    with molecule_selector_list_output:
        clear_output()
        display(molecule_selector_list)

def select_material_radio_change(change):
    material_title = material_selection_radio_button.value
    if material_title == "No material":
        with material_selector:
            clear_output()
            material_selector.dropdown.value = -1
            return
    
    material_selector.details_textbox.value = ''
    for object_type in CONFIG["slabs_concepts"]:
        object_schema = DATA_MODEL["classes"][object_type]
        object_title = object_schema["title"]
        if material_title == object_title:
            material_type = object_type
            
    with material_selector:
        clear_output()
        display_list = [
            material_selector.dropdown_boxes,
            ipw.HBox([material_selector.details_textbox, material_selector.image_box])
        ]
        material_selector.load_dropdown_box(material_type)
        display(ipw.VBox(display_list))


# Create reaction product concept

## Select molecule concept

In [ ]:
display(molecule_selector_list_output)
with molecule_selector_list_output:
    display(molecule_selector_list)  
display(add_remove_molecule_buttons_hbox)

## Select slab concept

In [ ]:
display(material_selection_radio_button)
display(material_selector)

## Properties

In [ ]:
display(product_widgets)

## Upload CDXML

In [ ]:
display(product_cdxml)

## Support files

In [ ]:
display(product_support_files)

## Save results

In [ ]:
display(save_close_buttons_hbox)
display(increase_buttons_size)
material_selection_radio_button.observe(select_material_radio_change, names='value')
material_selector.dropdown.observe(material_selector.load_metadata, names = 'value')
add_molecule_button.on_click(add_molecule_widget)
remove_molecule_button.on_click(remove_molecule_widget)
create_button.on_click(create_product_openbis)
quit_button.on_click(close_notebook)